Notes on things to do
- Fix exisitng analysis to be coherent
- Listings over time by region?
- Overall rental index over time. Properly organzied calculations
- Market rent by city
- Market rent by state
- Easier search and aggregation by state, market_name
- Rentals by market breakdown
- Rental growth by area
- Machine learning stuff
- Create utils

Breakdown:
There are already known factors to slice. Most likely a breakdown by state and zip codes and house size. The next breakdown would be which areas are increasing in price or something

notes
- date is when the data was collected, the same listings exist during multiple time instances. id are uuid used to identify each listing.

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json

import plotly.io as pio
pio.renderers.default='notebook'

In [16]:
raw_rental_df = pd.read_parquet('data/clean_housing_data.parquet')

rental_df = raw_rental_df.copy(deep= True)

initial_date = rental_df['date'].iloc[0]
initial_rental_df = rental_df[rental_df['date'] == initial_date]

unique_rental_df = rental_df[~rental_df['id'].duplicated(keep='last')]

In [4]:
rental_df.columns

Index(['id', 'unit_code', 'property_code', 'market_id', 'market_name',
       'address_1', 'address_2', 'city', 'state', 'zipcode', 'country', 'beds',
       'baths', 'sqft', 'market_rent', 'lat', 'lng', 'available_at', 'unit_id',
       'unit_status', 'is_syndicated', 'is_syndicated_ils', 'is_on_special',
       'new_construction', 'est_rehab_complete_date', 'rehab_type',
       'submarket_names', 'subdivision', 'bid_type', 'asset_review_type',
       'days_on_market', 'formattedAddress', 'date', 'has_virtual_tour',
       'btr_community', 'model_home'],
      dtype='object')

In [10]:
len(rental_df)

558806

In [30]:
# unique_rental_df[['country', 'state', 'market_name', 'submarket_names', 'city', 'zipcode']].value_counts()
unique_rental_df[['country', 'state', 'market_name', 'submarket_names', 'city', 'zipcode']].astype(str).describe(include= 'object').transpose()

,count,unique,top,freq
country,19197,1,US,19197
state,19197,12,FL,5941
market_name,19197,16,Atlanta,2456
submarket_names,19197,228,Colorado Springs,420
city,19197,869,Las Vegas,538
zipcode,19197,1753,34953.0,189


In [35]:
unique_rental_df[['beds', 'baths', 'sqft', 'market_rent']].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
beds,19197.0,3.505756,0.648809,1.00,3.0,3.0,4.0,7.0
baths,19197.0,2.314393,0.488892,0.25,2.0,2.0,2.5,7.0
sqft,19197.0,1893.365943,520.183183,180.00,1526.0,1810.0,2178.0,7928.0
market_rent,19197.0,2365.757152,518.666612,1110.00,1981.0,2285.0,2645.0,5492.0


In [39]:
unique_rental_df[['date', 'available_at', 'days_on_market']].head(5)

,date,available_at,days_on_market
152,2022-06-12 22:51:48,2022-06-07T00:00:00.000Z,4
853,2022-06-12 22:51:48,2022-06-21T00:00:00.000Z,0
1973,2022-06-12 22:51:48,2022-07-07T00:00:00.000Z,0
1978,2022-06-12 22:51:48,2022-07-15T00:00:00.000Z,0
2012,2022-06-13 07:22:05,2022-06-10T00:00:00.000Z,1


In [50]:
# unique_rental_df[['unit_status', 'is_syndicated', 'is_syndicated_ils', 'is_on_special', 'new_construction', 'rehab_type']].value_counts()
unique_rental_df[['unit_status', 'is_syndicated', 'is_syndicated_ils', 'is_on_special', 'new_construction', 'est_rehab_complete_date', 'rehab_type']].describe().transpose()

,count,unique,top,freq
unit_status,19197,3,Vacant Unrented Ready,10768
is_syndicated,19197,2,True,19164
is_syndicated_ils,19197,2,True,18574
is_on_special,19197,2,False,18677
new_construction,19197,2,False,18981
est_rehab_complete_date,12963,402,2022-08-12T00:00:00.000Z,129
rehab_type,19132,5,Turn,16528


In [49]:
# unique_rental_df[['subdivision', 'bid_type', 'asset_review_type', 'has_virtual_tour', 'btr_community', 'model_home']].value_counts()
unique_rental_df[['subdivision', 'bid_type', 'asset_review_type', 'has_virtual_tour', 'btr_community', 'model_home']].describe().transpose()

,count,unique,top,freq
subdivision,7702,5205,Port Malabar,64
bid_type,13406,6,Standard Sale,6917
asset_review_type,6668,7,RevEx,4082
has_virtual_tour,18458,2,False,9742
btr_community,5081,2,False,5071
model_home,5081,2,False,5079


In [9]:
date_listing_count = rental_df['date'].value_counts()

# Count of listings for each date. Tracks how much data and what days are covered by the data set
# There are some gaps in the data because I didn't feel like it
fig = go.Figure(
    go.Scatter(
        x= date_listing_count.index,
        y= date_listing_count.values,
        mode= 'markers'
    )
)

fig.update_layout(
    yaxis_title = 'Count of listings #'
)

fig.show()

In [11]:
# invh only really operates in a few states and zip codes
rental_df['state'].value_counts()
# rental_df[['state', 'zipcode']].value_counts()

FL    161598
AZ     82787
GA     61896
CO     47656
CA     45621
WA     38778
NC     37450
TX     35887
NV     28712
MN      9716
IL      8700
SC         5
Name: state, dtype: int64

In [12]:
# Trying to limit data set to only a time step for now
initialdate = rental_df['date'].iloc[0]
initialrental_df = rental_df[rental_df['date'] == initialdate]

In [13]:
initialrentbystate = initialrental_df.loc[:, ['state', 'market_rent']].groupby('state').mean()

In [14]:
rental_df[['state', 'zipcode']].value_counts()['CA']

zipcode
95843.0    2134
94533.0    1383
95687.0    1191
94509.0    1037
94585.0    1035
           ... 
92802.0       2
90712.0       2
95209.0       1
92078.0       1
92113.0       1
Length: 317, dtype: int64

In [38]:
cleaned_rental_df = rental_df.loc[:, ['date', 'id', 'formattedAddress', 'market_name', 'market_rent', 'days_on_market']].set_index(['date', 'id'])
date_indexs = cleaned_rental_df.index.unique(0)

In [68]:
# Finds the total change in rent between each day for the same listing id
rent_diff_dict = {}

for i in range(len(date_indexs)-1):
    # Finds difference in market_rent by id index
    market_rent_difference = cleaned_rental_df.loc[date_indexs[i+1], :]['market_rent'] - cleaned_rental_df.loc[date_indexs[i], :]['market_rent']
    rent_diff_dict[date_indexs[i+1]] = market_rent_difference.sum()

rent_diff_series = pd.Series(rent_diff_dict)

fig = go.Figure(
    go.Scatter(
        x= rent_diff_series.index,
        y= rent_diff_series.values
    )
)

fig.update_layout(
    title= "Same Property Rent Change",
    yaxis_title= "Rent Change",
)

fig.show()

In [81]:
# Not sure why this works
# This is finding the mean of every column even though market rent is the only one that matters
rental_by_market = cleaned_rental_df.groupby(['date', 'market_name'])[['market_rent', 'days_on_market']].mean()
rental_by_market_normalized = rental_by_market/rental_by_market.loc[date_indexs[0]]

# Group by date and market_name, normalize by initial date for each market_name, average normalized values
# This is effectively a normalized metric while correcting for market_name
rental_market_mean_normalized = rental_by_market_normalized.groupby(["date"]).mean()

In [91]:
# Breakdown by market_name

fig = px.line(rental_by_market, x=rental_by_market.index.get_level_values(0), y="market_rent", color=rental_by_market.index.get_level_values(1))
fig.show()

In [85]:
# Mean of all market_rents normalized to initial date

fig = go.Figure(
    go.Scatter(
        x= rental_market_mean_normalized.index,
        y= rental_market_mean_normalized['market_rent']
    )
)

fig.update_layout(
    yaxis_title= "Market Normalized Rent"
)

fig.show()

In [92]:
# Supposed to track how long a lisitng remains. I guess this is their internal display, haven't tried to double check it
# I don't even know how somethings have such low days on market, like things are apparenyl only there for a few days on southern california
fig = px.line(rental_by_market, x=rental_by_market.index.get_level_values(0), y="days_on_market", color=rental_by_market.index.get_level_values(1))
fig.show()